In [1]:
import numpy as np
import functools
import time
from tests import * # імпортуємо тести для функцій

## Еталони

In [2]:
zero = np.array([[1, 1, 1],
                 [1, 0, 1],
                 [1, 0, 1],
                 [1, 0, 1],
                 [1, 1, 1]])

one  = np.array([[0, 1, 0],
                 [0, 1, 0],
                 [0, 1, 0],
                 [0, 1, 0],
                 [0, 1, 0]])

two  = np.array([[1, 1, 1],
                 [0, 0, 1],
                 [1, 1, 1],
                 [1, 0, 0],
                 [1, 1, 1]])

three =np.array([[1, 1, 1],
                 [0, 0, 1],
                 [1, 1, 1],
                 [0, 0, 1],
                 [1, 1, 1]])

four = np.array([[1, 0, 1],
                 [1, 0, 1],
                 [1, 1, 1],
                 [0, 0, 1],
                 [0, 0, 1]])

five = np.array([[1, 1, 1],
                 [1, 0, 0],
                 [1, 1, 1],
                 [0, 0, 1],
                 [1, 1, 1]])

six =  np.array([[1, 1, 1],
                 [1, 0, 0],
                 [1, 1, 1],
                 [1, 0, 1],
                 [1, 1, 1]])

seven =np.array([[1, 1, 1],
                 [0, 0, 1],
                 [0, 1, 0],
                 [1, 0, 0],
                 [1, 0, 0]])

eight =np.array([[1, 1, 1],
                 [1, 0, 1],
                 [1, 1, 1],
                 [1, 0, 1],
                 [1, 1, 1]])


nine = np.array([[1, 1, 1],
                 [1, 0, 1],
                 [1, 1, 1],
                 [0, 0, 1],
                 [0, 0, 1]])

In [3]:
Y1 = [zero, one, two, three, four, five, six, seven, eight, nine] # еталонні зображення

In [4]:
Y = []
sc1 = 3    # наскільки збільшуємо висоту
sc2 = 3   # ... ширину
for i in range(len(Y1)):
    Y.append(upscale(Y1[i], sc1,sc2))

In [5]:
Y[0].shape

(15, 9)

## Генеруємо гістограми

In [6]:
np.random.seed(np.random.randint(0,100))
histogram = generate_histogram()
histogram

array([0.13452915, 0.06278027, 0.07772795, 0.0941704 , 0.13602392,
       0.13303438, 0.09715994, 0.10164425, 0.10612855, 0.0568012 ])

## Генеруємо вхідне зображення
### повертаємо зображення  і суму цифр

In [7]:
def task(Y, t, p):# генеруємо таск довжиною t з ймовірністю p
    r = np.random.choice(10, t, p=histogram)
    x_etalone = Y[r[0]]
    for i in range(1,t):
        x_etalone = np.concatenate((x_etalone,Y[r[i]]),axis=1)
    
    n = Y[0].shape[0]
    m = Y[0].shape[1]
    
    #print(sum(r))
    ksi = np.random.binomial(size=n*m*t, n=1, p=p).reshape((n,m*t))
    f = (x_etalone^ksi, sum(r)) 
    return f # повертаємо таск і суму чисел (ті що випали)

## Рахуємо ймовірності

In [8]:
def recog2(X_arr,p):
    probab= np.empty(10)                   # остаточні ймовірності
    d = (Y^X_arr)
    for ks in range(10):
        stepin = ( d- (Y[ks]^X_arr))
        if np.sum(stepin[ks])==0:
            h = np.zeros(10)
            h[ks] = 1
            return h
        stepin = [np.sum(i) for i in stepin]
        sum_ki = histogram*(p/(1-p))**np.array(stepin)
        probab[ks] = histogram[ks]/np.sum(sum_ki)                            # додаємо ймовірність
    return probab    

## Рекурсивно рахуємо ймов-ті для під-зображень

In [9]:
t = 21                 # кількість цифр
p = 0.1                  # рівень шуму 
v = task(Y, t,p)      # генеруємо таск
X = np.hsplit(v[0], t)      # розбиваємо таск по цифрах   

In [10]:
@functools.lru_cache(None)        # та ж функція що була раніше але з новими підрахунками
def f(t):
    if t==1:
        return recog2(X[t-1],p)    # умова виходу з рекурсії
    else:
        res = []                  # масив в якому будуть остаточні ймовірності
        z = recog2(X[t-1],p)
        q = f(t-1)
        for d in range(9*t+1):
            k2 = []
            
            for k in range(10):
                
                if k<=d and k>=d-9*(t-1):       
                    k2.append(z[k]*q[d-k]) # рахуємо рекурсивно   НЕЕЕЕ ТРОГАТЬ АААААААААААААААА 
            res.append(np.sum(k2))
    return res

## `Succes rate`

In [11]:
c=0
iterations = 10
for i in range(iterations):
    v = task(Y, t,p)      # генеруємо таск
    X = np.hsplit(v[0], t)      # розбиваємо таск по цифрах
    a = f(t)
    f.cache_clear()
    if (checking(a) == True and v[1]%3 == 0) or (checking(a) == False and v[1]%3 != 0):
        c += 1

In [12]:
print(str(c)+'/' + str(iterations ))

10/10


## `Для прикладу: 1000 чисел по 21 цифрі` 

In [13]:
f.cache_clear()

t = 21                 # кількість цифр
p = 0.1                  # рівень шуму 
Y = []
sc1 = 1    # наскільки збільшуємо висоту
sc2 = 1   # ... ширину
for i in range(len(Y1)):
    Y.append(upscale(Y1[i], sc1,sc2))

In [14]:
time1 = time.time()
for i in range(1000):
    v = task(Y, t,p)      # генеруємо таск
    X = np.hsplit(v[0], t)      # розбиваємо таск по цифрах
    a = f(t)
    f.cache_clear()
print(time.time()-time1)

40.99597930908203
